In [2]:
%pip install powerlaw 
import eikon as ek
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
import powerlaw



Note: you may need to restart the kernel to use updated packages.


In [3]:
df_close = pd.read_csv("../data/euro50_prices.csv")
df_esg = pd.read_csv("../data/euro50_esg.csv")

In [4]:
#Use daily close for correlation

if 'Date' in df_close.columns:
    df_close.set_index('Date', inplace=True)
df_close.index = pd.to_datetime(df_close.index)

df_pct = df_close.pct_change().dropna()
df_returns = np.log(1 + df_pct).dropna()
print(df_returns.head())

# Compute the correlation matrix
corr_matrix = df_returns.corr()

# Create heatmap using plotly
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix,
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    colorscale='RdBu_r',
    zmid=0,  # Center the colorscale at 0
))

# Update layout
fig.update_layout(
    title="Correlation Matrix of Euro Stoxx50 Stocks (log returns)",
    width=1000,
    height=1000,
    xaxis_tickangle=-45
)

fig.show()

# 3) (Optional) threshold the correlation matrix by some cutoff rho
rho = 0.6085859
adj_matrix = (corr_matrix.abs() > rho).astype(int)
np.fill_diagonal(adj_matrix.values, 0)

# adj_matrix is now a 0/1 adjacency matrix, where edges exist if |c_ij| > rho
print("Correlation Matrix:\n", corr_matrix)
print("Adjacency Matrix (|corr| > {}):\n".format(rho), adj_matrix)

C:\Users\wills\AppData\Local\Temp\ipykernel_63340\992289519.py:7: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_pct = df_close.pct_change().dropna()


             DHLn.DE   PERP.PA  NDAFI.HE   AIRP.PA    IBE.MC  SIEGn.DE  \
Date                                                                     
2023-01-03  0.007195 -0.002437  0.005871 -0.002522 -0.003184  0.008590   
2023-01-04  0.011379  0.010516  0.019133  0.026528  0.012226  0.032608   
2023-01-05  0.007469 -0.011330  0.017834 -0.004930 -0.004963 -0.003258   
2023-01-06  0.000676  0.010793  0.000000  0.026393  0.002867  0.009595   
2023-01-09  0.023122  0.008285  0.008613  0.008879 -0.005978  0.016031   

              SAN.MC  VOWG_p.DE   SAPG.DE   CRDI.MI  ...   RACE.MI   LVMH.PA  \
Date                                                 ...                       
2023-01-03  0.002629   0.016688  0.011127  0.033926  ...  0.006399  0.012456   
2023-01-04  0.036430   0.030816  0.021988  0.036701  ...  0.011222  0.048871   
2023-01-05  0.003706   0.009802  0.001389  0.007181  ... -0.000971 -0.013093   
2023-01-06  0.024086   0.008148  0.014766  0.012173  ...  0.014465  0.027622   
2

Correlation Matrix:
             DHLn.DE   PERP.PA  NDAFI.HE   AIRP.PA    IBE.MC  SIEGn.DE  \
DHLn.DE    1.000000  0.300299  0.340410  0.373146  0.232539  0.454035   
PERP.PA    0.300299  1.000000  0.217573  0.301623  0.212513  0.277783   
NDAFI.HE   0.340410  0.217573  1.000000  0.399614  0.178891  0.404066   
AIRP.PA    0.373146  0.301623  0.399614  1.000000  0.357040  0.406811   
IBE.MC     0.232539  0.212513  0.178891  0.357040  1.000000  0.169072   
SIEGn.DE   0.454035  0.277783  0.404066  0.406811  0.169072  1.000000   
SAN.MC     0.439200  0.235580  0.554135  0.390075  0.242514  0.418887   
VOWG_p.DE  0.400057  0.215078  0.358940  0.213876  0.120050  0.359162   
SAPG.DE    0.329401  0.256808  0.227776  0.387996  0.140176  0.386082   
CRDI.MI    0.340336  0.180450  0.446510  0.322039  0.163864  0.404224   
AD.AS      0.102516  0.205264  0.112664  0.222674  0.235203  0.108866   
BNPP.PA    0.379490  0.272737  0.610579  0.378681  0.237993  0.414429   
DTEGn.DE   0.177482  0.064794 

In [5]:
import networkx as nx
import plotly.graph_objects as go

def plot_network_graph(adj_matrix, sparse=0.1, largest_cc=False, drop_isolates=False):
    # Create a graph from the adjacency matrix
    G = nx.from_pandas_adjacency(adj_matrix)
    
    # Remove self-loops
    G.remove_edges_from(nx.selfloop_edges(G))

    if largest_cc:
        cc = max(nx.connected_components(G), key=len)
        G = G.subgraph(cc).copy()

    if drop_isolates:
        G = nx.subgraph(G, [node for node, degree in dict(G.degree()).items() if degree > 0]).copy()

    # Generate positions for all nodes
    communities = nx.community.greedy_modularity_communities(G)

    supergraph = nx.cycle_graph(len(communities))
    superpos = nx.spring_layout(supergraph, k=sparse/np.sqrt(len(G.nodes())),scale=4, seed=429)

    # Use the "supernode" positions as the center of each node cluster
    centers = list(superpos.values())
    pos = {}
    for center, comm in zip(centers, communities):
        pos.update(nx.spring_layout(nx.subgraph(G, comm), center=center))


    # Create edges
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=0.5, color='#888'),
        hoverinfo='none',
        mode='lines'
    )


    # Create nodes and name them by their labels
    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)



    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode="markers+text",
        hoverinfo='text',
        textposition="bottom center",
        text=list(G.nodes()),
        marker=dict(
            showscale=True,
            colorscale='YlGnBu',
            reversescale=True,
            color=[],
            size=10,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right'
            ),
            line_width=2
        )
    )

    # Color node points by the number of connections
    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(f'{list(G.nodes())[node]}<br>C: ' + str(len(adjacencies[1])))
    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    # Create network graph
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        #include rho in title to two decimal places
                        title=f'Financial Network Graph of EuroStoxx 50 Stocks, rho={rho:.4f}',
                        titlefont_size=16,
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        annotations=[dict(
                            text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                            showarrow=False,
                            xref="paper", yref="paper",
                            x=0.005, y=-0.002
                        )],
                        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                    )
    )

    fig.update_layout(
    # update width and height of the figure
    width=1000,
    height=800,
    # update the layout margin
)

    fig.show()




# Example usage
plot_network_graph(adj_matrix, sparse=1, largest_cc=False, drop_isolates=True)

In [6]:

# fit using linear regression with scipy
from scipy.optimize import curve_fit

# Flatten adjacency to degrees
G_price = nx.from_pandas_adjacency(adj_matrix)
G_price.remove_edges_from(nx.selfloop_edges(G_price))

degrees = [deg for (_, deg) in G_price.degree()]


# Max degree so we know how far to compute the histogram
max_degree = max(degrees)

# Histogram of degrees: hist[k] = number of nodes with degree k
hist = np.bincount(degrees)

# Probability p(k) for each degree k
p_k = hist / hist.sum()

print(f"p(k) for each degree k: {p_k}")

def exponential_degree(k, alpha, gamma):
    return alpha * np.exp(-gamma * k)

k_values = np.arange(1, max_degree + 1)   
p_values = p_k[1:]  


# Filter out any zero-probability bins so curve_fit doesn’t choke
nonzero_indices = np.where(p_values > 0)
k_fit = k_values[nonzero_indices]
p_fit = p_values[nonzero_indices]

# Now do the fit
popt, pcov = curve_fit(exponential_degree, k_fit, p_fit, p0=(1.0, 0.1))
alpha_fit, gamma_fit = popt
print("Fitted alpha =", alpha_fit)
print("Fitted gamma =", gamma_fit)

# Evaluate the fitted exponential at each k
p_exp_fit = exponential_degree(k_values, alpha_fit, gamma_fit)

# Compute sum of absolute deviations
e_fitting = np.sum(np.abs(p_k[1:] - p_exp_fit))  # ignoring k=0 if you like
print("Fitting error =", e_fitting)

# Create plotly figure
fig = go.Figure()

# Add empirical data points
fig.add_trace(go.Scatter(
    x=k_fit,
    y=p_fit,
    mode='markers+lines',
    opacity=0.7,
    name='Empirical p(k)',
    line=dict(color='blue')
))

# Add fitted curve
fig.add_trace(go.Scatter(
    x=k_fit,
    y=exponential_degree(k_fit, alpha_fit, gamma_fit),
    mode='lines',
    name=r'αe^(-γk) (fit)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title=f'Degree Distribution with Exponential Fit (α={alpha_fit:.2f}, γ={gamma_fit:.2f}), (rho={rho:.4f})',
    xaxis_title='Degree k',
    yaxis_title='p(k)',
    xaxis_type='log',
    yaxis_type='log',
    
)

fig.show()





p(k) for each degree k: [0.48 0.34 0.12 0.06]
Fitted alpha = 0.8910461319256052
Fitted gamma = 0.9682120263729733
Fitting error = 0.02131914470809322


In [7]:
import numpy as np
import networkx as nx
from scipy.optimize import curve_fit


max_ccs = []
errors = []
nodes = []


def exponential_degree(k, alpha, gamma):
    return alpha * np.exp(-gamma * k)

def make_graph(rho):
    # 1. Threshold the correlation matrix
    adj_matrix = (corr_matrix.abs() > rho).astype(int)
    np.fill_diagonal(adj_matrix.values, 0)

    # 2. Create a graph from the adjacency matrix
    G = nx.from_pandas_adjacency(adj_matrix)
    G.remove_edges_from(nx.selfloop_edges(G))

    cc = max(nx.connected_components(G), key=len)
    max_ccs.append(len(cc))

    return G

def fitting_error_for_threshold(rho):
    # 1. Build the graph at threshold rho
    G = make_graph(rho)

    #Count the number of connected components and store it in the nodes list
    nodes.append(len(list(nx.connected_components(G))))


    max_cc = max(nx.connected_components(G), key=len)
    

    # 2. Degree distribution
    degrees = [deg for _, deg in G.degree()]
    hist = np.bincount(degrees)
    p_k = hist / hist.sum()

    # 3. Fit alpha, gamma
    k_vals = np.arange(1, len(p_k))
    p_vals = p_k[1:]
    nonzero = p_vals > 0
    k_fit = k_vals[nonzero]
    p_fit = p_vals[nonzero]
    popt, _ = curve_fit(exponential_degree, k_fit, p_fit, p0=(1.0, 0.1))
    alpha_fit, gamma_fit = popt

    # 4. Compute fitting error
    p_exp_fit = exponential_degree(k_vals, alpha_fit, gamma_fit)
    e_fit = np.sum(np.abs(p_k[1:] - p_exp_fit))

    errors.append(e_fit)

    #make sure there are at least 10 nodes
    if len(max_cc) < 10:
        raise Warning(f"Graph has less than 10 nodes at rho = {rho}")
    return e_fit, alpha_fit, gamma_fit, max_cc

import warnings

print(nodes)

# Loop over candidate rho’s
rhos = np.linspace(0.4, 0.99, 100)
best_rho = None
best_err = np.inf
rhos_run = []
problems = []
for rho in rhos:
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            err, alpha_fit, gamma_fit, max_cc = fitting_error_for_threshold(rho)
        rhos_run.append(rho)
        
        if err < best_err:
            best_rho = rho
            best_err = err
    except:
        problems.append(rho)
        continue
    
# Plot the error as a function of rho
fig = go.Figure()
fig.add_trace(go.Scatter(x=rhos, y=errors, mode='lines', name='Fitting Error'))
fig.update_layout(
    title='Fitting Error vs. Threshold rho in Financial Network',
    xaxis_title='Threshold rho',
    yaxis_title='Fitting Error',
    width=1000,
    height=400
)
fig.show()

# Plot max connected component size
fig = go.Figure()
fig.add_trace(go.Scatter(x=rhos, y=max_ccs[:93], mode='lines', name='Max CC Size'))
fig.update_layout(
    title='Max Connected Component Size vs. Threshold rho in Financial Network',
    xaxis_title='Threshold rho',
    yaxis_title='Max CC Size',
    width=1000,
    height=400
)
fig.show()


print(f"Problems at thresholds: {problems}")

print(f"Best threshold = {best_rho}, with error = {best_err}")


[]


Problems at thresholds: [np.float64(0.6085858585858586), np.float64(0.6145454545454545), np.float64(0.6205050505050504), np.float64(0.6264646464646465), np.float64(0.6324242424242424), np.float64(0.6383838383838384), np.float64(0.6443434343434343), np.float64(0.6503030303030303), np.float64(0.6562626262626263), np.float64(0.6622222222222223), np.float64(0.6681818181818182), np.float64(0.6741414141414142), np.float64(0.6801010101010101), np.float64(0.686060606060606), np.float64(0.692020202020202), np.float64(0.6979797979797979), np.float64(0.7039393939393939), np.float64(0.7098989898989898), np.float64(0.7158585858585859), np.float64(0.7218181818181818), np.float64(0.7277777777777777), np.float64(0.7337373737373738), np.float64(0.7396969696969697), np.float64(0.7456565656565657), np.float64(0.7516161616161616), np.float64(0.7575757575757576), np.float64(0.7635353535353535), np.float64(0.7694949494949495), np.float64(0.7754545454545454), np.float64(0.7814141414141413), np.float64(0.7873

In [8]:
G = nx.from_pandas_adjacency(adj_matrix)    
G.remove_edges_from(nx.selfloop_edges(G))
cc = max(nx.connected_components(G), key=len)
G = G.subgraph(cc).copy()

# Calculate metrics
number_of_nodes = nx.number_of_nodes(G)
number_of_edges = nx.number_of_edges(G)
average_shortest_path_length = nx.average_shortest_path_length(G) if nx.is_connected(G) else None
diameter = nx.diameter(G) if nx.is_connected(G) else None
average_clustering = nx.average_clustering(G)
average_degree = sum(dict(G.degree()).values()) / number_of_nodes
mean_fitting_error = best_err

# Display results
results = {
    "Number of Nodes": number_of_nodes,
    "Number of Edges": number_of_edges,
    "Average Shortest Path Length": average_shortest_path_length,
    "Diameter": diameter,
    "Average Clustering Coefficient": average_clustering,
    "Average Degree": average_degree,
    "Mean Fitting Error": mean_fitting_error,
}

for key, value in results.items():
    print(f"{key}: {value}")
    

Number of Nodes: 7
Number of Edges: 6
Average Shortest Path Length: 2.1904761904761907
Diameter: 4
Average Clustering Coefficient: 0.0
Average Degree: 1.7142857142857142
Mean Fitting Error: 0.05226060499798757


In [9]:
# Create subplot figure
rho_values = [0, 0.3, 0.84222, 0.957, 0.961, 0.978485]

fig = make_subplots(rows=2, cols=3, subplot_titles=[f"rho = {rho}" for rho in rho_values])

# Generate subplots
for i, rho in enumerate(rho_values):
    adj_matrix = (corr_matrix.abs() > rho).astype(int)
    G = nx.from_pandas_adjacency(adj_matrix)
    G.remove_edges_from(nx.selfloop_edges(G))
    G.remove_nodes_from(list(nx.isolates(G)))
    communities = nx.community.greedy_modularity_communities(G)
    supergraph = nx.cycle_graph(len(communities))
    superpos = nx.spring_layout(G, scale=10, seed=429)
    centers = list(superpos.values())
    pos = {}
    for center, comm in zip(centers, communities):
        pos.update(nx.spring_layout(nx.subgraph(G, comm), center=center))
    # pos = nx.spring_layout(G)
    
    edge_x = []
    edge_y = []
    for edge in G.edges():
        x0, y0 = pos[edge[0]]
        x1, y1 = pos[edge[1]]
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        #set the length of edge to be inverse to correlation between the two nodes
        line=dict(width=0.5, color='#888'),
        hoverinfo='text',
        mode='lines',
        text=[f"Correlation: {corr_matrix.loc[edge[0], edge[1]]}" for edge in G.edges()]
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)

    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers',
        hoverinfo='text',
        marker=dict(
            showscale=(i == 0),  # Only show colorbar in the first subplot
            colorscale='Viridis',
            reversescale=True,
            color=[],
            size=10,
            cmin=0,
            cmax=49,
            colorbar=dict(
                thickness=15,
                title='Node Connections',
                xanchor='left',
                titleside='right',
                x=1.05  # Adjust this to position the colorbar correctly
            ),
            line_width=2
        )
    )

    node_adjacencies = []
    node_text = []
    for node, adjacencies in enumerate(G.adjacency()):
        node_adjacencies.append(len(adjacencies[1]))
        node_text.append(f'{list(G.nodes())[node]}<br>connections: ' + str(len(adjacencies[1])))
    node_trace.marker.color = node_adjacencies
    node_trace.text = node_text

    row = i // 3 + 1
    col = i % 3 + 1
    fig.add_trace(edge_trace, row=row, col=col)
    fig.add_trace(node_trace, row=row, col=col)

fig.update_layout(
    height=800, width=1400, 
    title_text="Financial Network Graphs with Different rho Values",
    showlegend=False  # Hide the legend
)
fig.show()


In [11]:
import pandas as pd
import numpy as np

# Normalize the daily close prices p_j(t) to [0, 1] Using the formula:  p_j(t) = x1_j(t) / max_t x1_j(t).


def normalize_prices(df_prices):
    """Normalize each column (stock) by dividing by its column max."""
    df_norm = df_prices.div(df_prices.max(axis=0), axis=1)
    return df_norm


# 3) Normalize the ESG scores to [0.1, 1] 

def normalize_esg(esg_series):

    esg_min = esg_series.min()
    esg_max = esg_series.max()

    norm_esg = 1.0 - ((esg_max - esg_series) / (esg_max - esg_min)) * 0.9
    return norm_esg


# 4) Compute the composite indicator: 


def compute_composite_indicator(df_prices, esg_series):


    esg_series_aligned = esg_series.reindex(df_prices.columns)
    
    # broadcast ESG across each row => same ESG each day
    esg_matrix = np.tile(esg_series_aligned.values, (df_prices.shape[0], 1))
    esg_df = pd.DataFrame(esg_matrix, index=df_prices.index, columns=df_prices.columns)

    # CI_j(t) = sqrt( p_j(t) * esg_j )
    ci_df = np.sqrt(df_prices * esg_df)
    
    return ci_df

# 5) Putting it all together


def incorporate_esg_and_prices(df_prices, df_esg_single):
 
    # 1) Price normalization
    df_prices_norm = normalize_prices(df_prices)
    
    # 2) ESG normalization
    esg_norm = normalize_esg(df_esg_single)
    
    # 3) Composite
    CI = compute_composite_indicator(df_prices_norm, esg_norm)
    
    return CI

def convert_esg_data_to_series(df_esg):
  
    esg_series = df_esg.set_index('Instrument')['ESG_Score']
    return esg_series




In [12]:
df_esg = pd.read_csv("../data/euro50_esg.csv")

CI_price = incorporate_esg_and_prices(df_returns, convert_esg_data_to_series(df_esg))
    
print("Normalized composite indicator (first 5 rows):")
print(CI_price.head())

Normalized composite indicator (first 5 rows):
             DHLn.DE   PERP.PA  NDAFI.HE   AIRP.PA    IBE.MC  SIEGn.DE  \
Date                                                                     
2023-01-03  0.247006       NaN  0.238239       NaN       NaN  0.321631   
2023-01-04  0.310619  0.214519  0.430083  0.512061  0.570441  0.626631   
2023-01-05  0.251666       NaN  0.415227       NaN       NaN       NaN   
2023-01-06  0.075726  0.217325  0.000000  0.510759  0.276262  0.339917   
2023-01-09  0.442786  0.190410  0.288557  0.296246       NaN  0.439369   

              SAN.MC  VOWG_p.DE   SAPG.DE   CRDI.MI  ...   RACE.MI   LVMH.PA  \
Date                                                 ...                       
2023-01-03  0.200188   0.358950  0.368080  0.442998  ...  0.088143  0.266649   
2023-01-04  0.745259   0.487782  0.517431  0.460760  ...  0.116725  0.528170   
2023-01-05  0.237687   0.275108  0.130065  0.203815  ...       NaN       NaN   
2023-01-06  0.605976   0.250819  0

c:\Users\wills\Desktop\IFTE0023\Project\.conda\Lib\site-packages\pandas\core\internals\blocks.py:393: RuntimeWarning:

invalid value encountered in sqrt



In [13]:
# calculate the correlation matrix of the composite indicator
corr_matrix_ci = CI_price.corr()



# Create heatmap using plotly
# Create heatmap using plotly
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
    z=corr_matrix_ci,
    x=corr_matrix_ci.columns,
    y=corr_matrix_ci.columns,
    colorscale='RdBu_r',
    zmid=0,  # Center the colorscale at 0
))

# Update layout
fig.update_layout(
    title="Correlation Matrix of Euro Stoxx50 Stocks (log returns)",
    width=1000,
    height=1000,
    xaxis_tickangle=-45
)

fig.show()

# 3) (Optional) threshold the correlation matrix by some cutoff rho
rho = 0.9612121212121212
adj_matrix_ci = (corr_matrix_ci.abs() > rho).astype(int)
np.fill_diagonal(adj_matrix_ci.values, 0)

# adj_matrix is now a 0/1 adjacency matrix, where edges exist if |c_ij| > rho
print("Correlation Matrix:\n", corr_matrix_ci)
print("Adjacency Matrix (|corr| > {}):\n".format(rho), adj_matrix_ci)


Correlation Matrix:
             DHLn.DE   PERP.PA  NDAFI.HE   AIRP.PA    IBE.MC  SIEGn.DE  \
DHLn.DE    1.000000  0.136943  0.349354  0.281551  0.104128  0.321256   
PERP.PA    0.136943  1.000000  0.075517  0.209613 -0.031046  0.149983   
NDAFI.HE   0.349354  0.075517  1.000000  0.161831  0.175936  0.274308   
AIRP.PA    0.281551  0.209613  0.161831  1.000000  0.260314  0.360110   
IBE.MC     0.104128 -0.031046  0.175936  0.260314  1.000000  0.160370   
SIEGn.DE   0.321256  0.149983  0.274308  0.360110  0.160370  1.000000   
SAN.MC     0.390364  0.112263  0.410992  0.223081  0.222832  0.198803   
VOWG_p.DE  0.302891  0.114955  0.193759  0.135604  0.113146  0.318716   
SAPG.DE    0.341186  0.258483  0.073036  0.294595  0.087549  0.278041   
CRDI.MI    0.197764  0.188911  0.444676  0.190016  0.125850  0.215775   
AD.AS      0.077370  0.056454  0.162610  0.225095  0.077191  0.155900   
BNPP.PA    0.234770  0.265533  0.390951  0.179742  0.213990  0.207447   
DTEGn.DE   0.263102 -0.041455 

In [14]:
import numpy as np
import networkx as nx
from scipy.optimize import curve_fit

max_ccs = []
errors = []
num_nodes_without_isolates = []


def exponential_degree(k, alpha, gamma):
    return alpha * np.exp(-gamma * k)

def make_graph(rho):
    # 1. Threshold the correlation matrix
    adj_matrix_ci = (corr_matrix_ci.abs() > rho).astype(int)
    np.fill_diagonal(adj_matrix_ci.values, 0)

    # 2. Create a graph from the adjacency matrix
    G = nx.from_pandas_adjacency(adj_matrix_ci)
    G.remove_edges_from(nx.selfloop_edges(G))

    count_non_isolates = 0
    for node, degree in dict(G.degree()).items():
        if degree > 0:
            count_non_isolates += 1

    num_nodes_without_isolates.append(count_non_isolates)

    cc = max(nx.connected_components(G), key=len)
    max_ccs.append(len(cc))

    return G

def fitting_error_for_threshold(rho):
    # 1. Build the graph at threshold rho
    G = make_graph(rho)

    max_cc = max(nx.connected_components(G), key=len)

    # 2. Degree distribution
    degrees = [deg for _, deg in G.degree()]
    hist = np.bincount(degrees)
    p_k = hist / hist.sum()

    # 3. Fit alpha, gamma
    k_vals = np.arange(1, len(p_k))
    p_vals = p_k[1:]
    nonzero = p_vals > 0
    k_fit = k_vals[nonzero]
    p_fit = p_vals[nonzero]
    popt, _ = curve_fit(exponential_degree, k_fit, p_fit, p0=(1.0, 0.1))
    alpha_fit, gamma_fit = popt

    # 4. Compute fitting error
    p_exp_fit = exponential_degree(k_vals, alpha_fit, gamma_fit)
    e_fit = np.sum(np.abs(p_k[1:] - p_exp_fit))

    errors.append(e_fit)

    if len(max_cc) < 2:
        raise Warning(f"Graph has less than 10 nodes at rho = {rho}")
    return e_fit, alpha_fit, gamma_fit, max_cc

import warnings

# Loop over candidate rho’s
rhos = np.linspace(0.3, 0.99, 100)
best_rho = None
best_err = float('inf')
rhos_run = []
problems = []
for rho in rhos:
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            err, alpha_fit, gamma_fit, max_cc = fitting_error_for_threshold(rho)
        rhos_run.append(rho)
        
        if err < best_err:
            best_rho = rho
            best_err = err
    except:
        problems.append(rho)
        continue
    
# Plot the error as a function of rho
fig = go.Figure()
fig.add_trace(go.Scatter(x=rhos, y=errors, mode='lines', name='Fitting Error'))
fig.update_layout(
    title='Fitting Error vs. Threshold rho',
    xaxis_title='Threshold rho',
    yaxis_title='Fitting Error',
    width=1000,
    height=400
)
fig.show()

# Plot max connected component size
fig = go.Figure()
fig.add_trace(go.Scatter(x=rhos, y=max_ccs[:93], mode='lines', name='Max CC Size'))
fig.update_layout(
    title='Max Connected Component Size vs. Threshold rho',
    xaxis_title='Threshold rho',
    yaxis_title='Max CC Size',
    width=1000,
    height=400
)
fig.show()

# Plot number of nodes without isolates
fig = go.Figure()
fig.add_trace(go.Scatter(x=rhos, y=num_nodes_without_isolates, mode='lines', name='Nodes without Isolates'))
fig.update_layout(
    title='Number of Nodes without Isolates vs. Threshold rho',
    xaxis_title='Threshold rho',
    yaxis_title='Number of Nodes without Isolates',
    width=1000,
    height=400
)
fig.show()


print(f"Problems at thresholds: {problems}")

print(f"Best threshold = {best_rho}, with error = {best_err}")


Problems at thresholds: [np.float64(0.6066666666666667), np.float64(0.6136363636363635), np.float64(0.6206060606060606), np.float64(0.6275757575757576), np.float64(0.6345454545454545), np.float64(0.6415151515151515), np.float64(0.6484848484848484), np.float64(0.6554545454545455), np.float64(0.6624242424242424), np.float64(0.6693939393939394), np.float64(0.6763636363636363), np.float64(0.6833333333333333), np.float64(0.6903030303030302), np.float64(0.6972727272727273), np.float64(0.7042424242424242), np.float64(0.7112121212121212), np.float64(0.7181818181818181), np.float64(0.7251515151515151), np.float64(0.7321212121212122), np.float64(0.739090909090909), np.float64(0.7460606060606061), np.float64(0.7530303030303029), np.float64(0.76), np.float64(0.766969696969697), np.float64(0.7739393939393939), np.float64(0.7809090909090909), np.float64(0.7878787878787878), np.float64(0.7948484848484848), np.float64(0.8018181818181818), np.float64(0.8087878787878788), np.float64(0.8157575757575757),

In [20]:
from scipy.optimize import curve_fit

rho = 0.53
adj_matrix_ci = (corr_matrix_ci.abs() > rho).astype(int)

# Flatten adjacency to degrees
CI_price = nx.from_pandas_adjacency(adj_matrix_ci)
CI_price.remove_edges_from(nx.selfloop_edges(CI_price))

degrees = [deg for (_, deg) in CI_price.degree()]


# Max degree so we know how far to compute the histogram
max_degree = max(degrees)

# Histogram of degrees: hist[k] = number of nodes with degree k
hist = np.bincount(degrees)

# Probability p(k) for each degree k
p_k = hist / hist.sum()

print(f"p(k) for each degree k: {p_k}")

def exponential_degree(k, alpha, gamma):
    return alpha * np.exp(-gamma * k)

k_values = np.arange(1, max_degree + 1)   # degrees from 1..max_degree
p_values = p_k[1:]  # skip p_k[0] if needed


# Filter out any zero-probability bins so curve_fit doesn’t choke
nonzero_indices = np.where(p_values > 0)
k_fit = k_values[nonzero_indices]
p_fit = p_values[nonzero_indices]

# Now do the fit
popt, pcov = curve_fit(exponential_degree, k_fit, p_fit, p0=(1.0, 0.1))
alpha_fit, gamma_fit = popt
print("Fitted alpha =", alpha_fit)
print("Fitted gamma =", gamma_fit)

# Evaluate the fitted exponential at each k
p_exp_fit = exponential_degree(k_values, alpha_fit, gamma_fit)

# Compute sum of absolute deviations
e_fitting = np.sum(np.abs(p_k[1:] - p_exp_fit))  # ignoring k=0 if you like
print("Fitting error =", e_fitting)


# Create plotly figure
fig = go.Figure()

# Add empirical data points
fig.add_trace(go.Scatter(
    x=k_fit,
    y=p_fit,
    mode='markers',
    name='Empirical p(k)',
))

# Add fitted curve
fig.add_trace(go.Scatter(
    x=k_fit,
    y=exponential_degree(k_fit, alpha_fit, gamma_fit),
    mode='lines',
    name=r'αe^(-γk) (fit)',
    line=dict(color='red')
))

# Update layout
fig.update_layout(
    title='Degree Distribution with Exponential Fit',
    xaxis_title='Degree k',
    yaxis_title='p(k)',
    xaxis_type='log',
    yaxis_type='log',
    
)

fig.show()

p(k) for each degree k: [0.78 0.12 0.1 ]
Fitted alpha = 0.144
Fitted gamma = 0.18232155679395456
Fitting error = 0.0


C:\Users\wills\AppData\Local\Temp\ipykernel_63340\1788625808.py:37: OptimizeWarning:

Covariance of the parameters could not be estimated



In [21]:

G = nx.from_pandas_adjacency(adj_matrix_ci)
G.remove_edges_from(nx.selfloop_edges(G))
cc = max(nx.connected_components(G), key=len)
G = G.subgraph(cc).copy()

# Calculate metrics
number_of_nodes = nx.number_of_nodes(G)
number_of_edges = nx.number_of_edges(G)
average_shortest_path_length = nx.average_shortest_path_length(G) if nx.is_connected(G) else None
diameter = nx.diameter(G) if nx.is_connected(G) else None
average_clustering = nx.average_clustering(G)
average_degree = sum(dict(G.degree()).values()) / number_of_nodes
mean_fitting_error = best_err

# Display results
results = {
    "Number of Nodes": number_of_nodes,
    "Number of Edges": number_of_edges,
    "Average Shortest Path Length": average_shortest_path_length,
    "Diameter": diameter,
    "Average Clustering Coefficient": average_clustering,
    "Average Degree": average_degree,
    "Mean Fitting Error": mean_fitting_error,
    "Modulartiy": nx.community.modularity(G, nx.community.greedy_modularity_communities(G))
}

for key, value in results.items():
    print(f"{key}: {value}")
    

Number of Nodes: 4
Number of Edges: 3
Average Shortest Path Length: 1.6666666666666667
Diameter: 3
Average Clustering Coefficient: 0.0
Average Degree: 1.5
Mean Fitting Error: 0.0
Modulartiy: 0.16666666666666663


In [195]:
adj_matrix = (corr_matrix.abs() > 0.65).astype(int)
plot_network_graph(adj_matrix, sparse=0.7, largest_cc=False, drop_isolates=True)

G_SelectedFN = nx.from_pandas_adjacency(adj_matrix)
G_SelectedFN.remove_edges_from(nx.selfloop_edges(G_SelectedFN))
G_SelectedFN.remove_nodes_from(list(nx.isolates(G_SelectedFN)))

# print nodes names
print(G_SelectedFN.nodes())
print(len(G_SelectedFN.nodes()))

['SIEGn.DE', 'SAN.MC', 'CRDI.MI', 'BNPP.PA', 'BBVA.MC', 'ALVG.DE', 'HRMS.PA', 'BMWG.DE', 'TTEF.PA', 'ENI.MI', 'PRTP.PA', 'SCHN.PA', 'LVMH.PA', 'MBGn.DE', 'AXAF.PA', 'INGA.AS', 'ISP.MI']
17


In [194]:
adj_matrix_ci = (corr_matrix.abs() > 0.7).astype(int)

plot_network_graph(adj_matrix_ci, sparse=0.7, largest_cc=False, drop_isolates=True)

G_SelectedESG = nx.from_pandas_adjacency(adj_matrix_ci)
G_SelectedESG.remove_edges_from(nx.selfloop_edges(G_SelectedESG))
G_SelectedESG.remove_nodes_from(list(nx.isolates(G_SelectedESG)))

# print nodes names
print(G_SelectedESG.nodes())
print(len(G_SelectedESG.nodes()))

['SAN.MC', 'CRDI.MI', 'BNPP.PA', 'BBVA.MC', 'HRMS.PA', 'BMWG.DE', 'TTEF.PA', 'ENI.MI', 'LVMH.PA', 'MBGn.DE', 'INGA.AS', 'ISP.MI']
12


## Index Calculation

In [26]:
market_cap = pd.read_csv("../data/euro50_marketcap.csv")
if 'Date' in market_cap.columns:
    market_cap.set_index('Date', inplace=True)
market_cap.index = pd.to_datetime(market_cap.index)
market_cap.head()

,ABI.BR,AD.AS,ADSGn.DE,ADYEN.AS,AIR.PA,AIRP.PA,ALVG.DE,ASML.AS,AXAF.PA,BASFn.DE,...,SAPG.DE,SASY.PA,SCHN.PA,SGEF.PA,SGOB.PA,SIEGn.DE,STLAM.MI,TTEF.PA,VOWG_p.DE,WLSNc.AS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-02,9.884618e+10,2.687032e+10,2.298600e+10,4.047476e+10,8.974502e+10,7.066200e+10,8.189291e+10,2.083813e+11,6.228587e+10,4.289163e+10,...,1.196809e+11,1.162034e+11,7.580687e+10,5.596233e+10,2.421998e+10,1.103470e+11,4.385610e+10,1.572003e+11,7.022624e+10,2.528809e+10
2023-01-03,9.874195e+10,2.719825e+10,2.373840e+10,4.052434e+10,9.028100e+10,7.048404e+10,8.276003e+10,2.097520e+11,6.269742e+10,4.370057e+10,...,1.210200e+11,1.170273e+11,7.646934e+10,5.589749e+10,2.485437e+10,1.112990e+11,4.442808e+10,1.544764e+11,7.101164e+10,2.534474e+10
2023-01-04,1.007918e+11,2.747153e+10,2.490840e+10,4.107544e+10,9.094309e+10,7.237882e+10,8.530091e+10,2.164844e+11,6.360284e+10,4.555085e+10,...,1.237104e+11,1.169512e+11,7.983879e+10,5.721772e+10,2.596325e+10,1.149880e+11,4.560418e+10,1.508620e+11,7.281740e+10,2.556105e+10
2023-01-05,9.926311e+10,2.718335e+10,2.493720e+10,3.979261e+10,9.080122e+10,7.202676e+10,8.451445e+10,2.176535e+11,6.289732e+10,4.614079e+10,...,1.238824e+11,1.145176e+11,7.973599e+10,5.741222e+10,2.649448e+10,1.146140e+11,4.607976e+10,1.517525e+11,7.332393e+10,2.512328e+10
2023-01-06,9.978427e+10,2.720322e+10,2.532240e+10,4.062304e+10,9.185741e+10,7.395305e+10,8.542190e+10,2.223299e+11,6.430918e+10,4.720448e+10,...,1.257251e+11,1.154936e+11,8.103809e+10,5.804286e+10,2.701539e+10,1.157190e+11,4.642038e+10,1.536906e+11,7.367117e+10,2.539624e+10


In [28]:
# Calculate the daily index based on percentage changes in market cap, sum in axis 1
market_cap_sum = market_cap.sum(axis=1)
market_index_pct = market_cap_sum.pct_change().add(1)
market_index_pct.head()

Date
2023-01-02         NaN
2023-01-03    1.007654
2023-01-04    1.027035
2023-01-05    0.997806
2023-01-06    1.015240
dtype: float64

In [29]:
# Plot the daily change in market cap
fig = go.Figure()

fig.add_trace(go.Scatter(x=market_index_pct.index, y=market_index_pct, mode='lines', 
                        name='Market Index',
                        line=dict(color='#f8c471 ', dash='solid')))

# selected rics
FNrics = ['SIEGn.DE', 'SAN.MC', 'CRDI.MI', 'BNPP.PA', 'BBVA.MC', 'ALVG.DE', 'HRMS.PA', 'BMWG.DE', 'TTEF.PA', 'ENI.MI', 'PRTP.PA', 'SCHN.PA', 'LVMH.PA', 'MBGn.DE', 'AXAF.PA', 'INGA.AS', 'ISP.MI']



#print number of selected rics
print(len(FNrics))
FN_esg_rics = ['SAN.MC', 'CRDI.MI', 'BNPP.PA', 'BBVA.MC', 'HRMS.PA', 'BMWG.DE', 'TTEF.PA', 'ENI.MI', 'LVMH.PA', 'MBGn.DE', 'INGA.AS', 'ISP.MI']
market_cap_selected = market_cap[FNrics]
esg_cap_selected = market_cap[FN_esg_rics]
market_cap_selected_sum = market_cap_selected.sum(axis=1)
esg_cap_selected_sum = esg_cap_selected.sum(axis=1)
market_index_selected = market_cap_selected_sum.pct_change().add(1)
esg_index_selected = esg_cap_selected_sum.pct_change().add(1)

fig.add_trace(go.Scatter(x=market_index_selected.index, y=market_index_selected, 
                        mode='lines',
                        name='Selected Stocks',
                        opacity=0.5,
                        line=dict(color='black', dash='solid')))

fig.add_trace(go.Scatter(x=esg_index_selected.index, y=esg_index_selected, 
                        mode='lines',
                        name='Selected Stocks',
                        opacity=0.5,
                        line=dict(color='red', dash='dash')))

fig.update_layout(
    title='Euro Stoxx50 Market Index',
    xaxis_title='Date',
    yaxis_title='Daily Return',
    width=1200,
    height=400,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()

# Plot the index 



17


In [30]:
#Calculate and plot the index for the selected stocks and the market index with the first stock as the base
base = market_cap_sum.iloc[0]
market_cap_sum = market_cap.sum(axis=1)
market_index = market_cap_sum / base

base = market_cap_selected_sum.iloc[0]
market_index_selected = market_cap_selected_sum / base
base = esg_cap_selected_sum.iloc[0]
esg_index_selected = esg_cap_selected_sum / base

fig = go.Figure()

fig.add_trace(go.Scatter(x=market_index.index, y=market_index, mode='lines',
                        name='Market Index',
                        opacity=0.5,
                        line=dict(color='black', dash='solid')))
fig.add_trace(go.Scatter(x=market_index_selected.index, y=market_index_selected, mode='lines',
                        name='Selected Stocks',
                        opacity=0.5,
                        line=dict(color='blue', dash='dash')))
fig.add_trace(go.Scatter(x=esg_index_selected.index, y=esg_index_selected, mode='lines',
                        name='ESG Network Stocks',
                        opacity=0.5,
                        line=dict(color='red', dash='solid')))
fig.update_layout(
    title='Euro Stoxx50 Market Index',
    xaxis_title='Date',
    yaxis_title='Daily Return',
    width=1200,
    height=400,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)


In [31]:
#plot the differences between the market index and the selected stocks index
fig = go.Figure()

fig.add_trace(go.Scatter(x=market_index_selected.index, y=market_index_selected - market_index, mode='lines',
                        name='Selected Stocks',
                        opacity=0.5,
                        line=dict(color='blue', dash='dash')))

fig.add_trace(go.Scatter(x=esg_index_selected.index, y=esg_index_selected - market_index, mode='lines',
                        name='ESG Network Stocks',
                        opacity=0.5,
                        line=dict(color='red', dash='solid')))

fig.update_layout(
    title='Euro Stoxx50 Market Index',
    xaxis_title='Date',
    yaxis_title='Daily Return Error',
    width=1200,
    height=400,
    legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
    )
)

fig.show()